In [1]:
from platform import python_version
print(python_version()) # tested with 3.7.1
# this adds the python wrapper module directory to the path.
import os
import sys
import importlib
bindingspath = os.getcwd() + '/../build/python'
sys.path.append(bindingspath)
print(sys.path)

3.7.1
['/home/adria/Documents/Documents/TUM/dlfs/libdl/python', '/home/adria/anaconda3/lib/python37.zip', '/home/adria/anaconda3/lib/python3.7', '/home/adria/anaconda3/lib/python3.7/lib-dynload', '', '/home/adria/anaconda3/lib/python3.7/site-packages', '/home/adria/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/home/adria/.ipython', '/home/adria/Documents/Documents/TUM/dlfs/libdl/python/../build/python']


In [2]:
import pybindings as pyb

In [3]:
# Load Mnist
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
mnist = fetch_openml('mnist_784', version=1, cache=True)
trX, teX, trY, teY = train_test_split(mnist.data / 255.0, mnist.target.astype("int0"), test_size = 0.33)

# Data vectors are expected in columns
trX = trX.transpose();
teX = teX.transpose();

In [4]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(categories='auto')
enc.fit(trY.reshape(-1, 1))

# Do one-hot encoding labels
trLabelsOneHot = enc.transform(trY.reshape(-1, 1)).toarray()
teLabelsOneHot = enc.transform(teY.reshape(-1, 1)).toarray()

In [10]:
print("Values per Image:", trX.shape[0])
print("Number of images for training:", trX.shape[1])
print("Number of images for testing:", teX.shape[1])
print("--- ---")
print("Training Labels:", trLabelsOneHot.shape[0])
print("Training Labels:", trLabelsOneHot.shape[1])

Values per Image: 784
Number of images for training: 46900
Number of images for testing: 23100
--- ---
Training Labels: 46900
Training Labels: 10


In [6]:
#subsample
trainSampleSize = 46900
trXSub = trX[:,0:trainSampleSize]
trLabelsOneHotSub = trLabelsOneHot[0:trainSampleSize,:]
testSampleSize = 20000;
teXSub = teX[:,0:testSampleSize]
teLabelsOneHotSub = teLabelsOneHot[0:testSampleSize,:]
print("Values per Image:", trXSub.shape[0])
print("Number of images for training:", trXSub.shape[1])
print("--- ---")
print("Training Labels:", trLabelsOneHotSub.shape[0])
print("Training Labels:", trLabelsOneHotSub.shape[1])

Values per Image: 784
Number of images for training: 46900
--- ---
Training Labels: 46900
Training Labels: 10


In [7]:
#importlib.reload(pyb)

In [8]:
exampleModel2 = pyb.ExampleModel()
exampleModel2.setTrainInputs(trXSub,1,28,28)
exampleModel2.setTrainLabels(trLabelsOneHotSub,10)
exampleModel2.setTestInputs(teXSub);
exampleModel2.setTestLabels(teLabelsOneHotSub)
exampleModel2.setLearningRate(0.005)


In [9]:
exampleModel2.runExample(2)

lossLayer.GetLoss() ++++++++++++++++++++++++++++++++++++++++++++++++++
0.0173987
Batch 0
lossLayer.GetLoss() ++++++++++++++++++++++++++++++++++++++++++++++++++
0.000472469
Batch 1
test Accuracy is 0.9413
